In [ ]:
# Import necessary libraries
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import User  # Replace 'your_module' with the appropriate module where the User class is defined

# Create an engine
engine = create_engine('sqlite:///instance/database.db')  # Replace 'example.db' with your database URI

# Create a Session
Session = sessionmaker(bind=engine)
session = Session()

# Query the User
user = session.query(User).all()

# Close the session after the query
session.close()

# Print the user details
if user:
    print(f"User found: {user}")
else:
    print("User not found.")


In [ ]:
from sqlalchemy import create_engine, MetaData

# Replace 'your_database_uri' with the appropriate URI for your database
engine = create_engine('sqlite:///instance/database.db') 

# Create a metadata object
metadata = MetaData()

# Reflect the database tables
metadata.reflect(bind=engine)

# Print the table names
print("Tables in the database:")
for table in metadata.tables.values():
    print(table.name)


In [6]:
import os
from dotenv import load_dotenv
from langchain import PromptTemplate
from src.bot_main import AIBot
from src.utils import read_input

load_dotenv()

inputs_dict = read_input(os.getenv("INPUT_FILE_PATH"))

template_str = inputs_dict["template"]

template = PromptTemplate.from_template(template_str)

ai_model = AIBot(model_name = os.getenv("LLM_MODEL_NAME"),
                 template=template)

In [7]:
user_input = "How do you make Elmer's slime less sticky?"
context = inputs_dict["context"]
convo_history = inputs_dict["convo_history"]

ai_model.ask_ai({"question": user_input, "context": context, "convo_history": convo_history})

Asking Model...
Data added to JSON file.


"To make Elmer's slime less sticky, you can add more activator (like contact lens solution or laundry detergent) into the slime. Adding a little bit at a time while kneading the slime will reduce its stickiness. However, be careful not to add too much as it can make the slime too hard."

In [6]:
import markdown

# Example Markdown text
markdown_text = """

This is normal text:
1. Dot point 1
2. Dot point 2

``` {html}
<body>
    <div class="container">
        <h1 class="jumbotron text-center">Sparks AI</h1>
        <div class="chat-container">
            <div class="chat-box" id="chatbox">
                <div class="bot-message">Hi! I am Sparks AI, how can I assist you today?</div>
            </div>
            <div class="user-input">
                <input id="textInput" class="form-control input-text" type="text" name="msg" placeholder="Type Your Message Here">
                <input id="buttonInput" class="btn btn-success input-button" type="button" value="Send">
            </div>
        </div>
    </div>
</body>
```
"""

# Convert Markdown to HTML
html = markdown.markdown(markdown_text, extensions=['markdown.extensions.fenced_code', 'markdown.extensions.codehilite'])

# Print the HTML
print(html)


<p>This is normal text:
1. Dot point 1
2. Dot point 2</p>
<div class="codehilite"><pre><span></span><code><span class="nt">&lt;body&gt;</span>
<span class="w">    </span><span class="nt">&lt;div</span><span class="w"> </span><span class="na">class=</span><span class="s">&quot;container&quot;</span><span class="nt">&gt;</span>
<span class="w">        </span><span class="nt">&lt;h1</span><span class="w"> </span><span class="na">class=</span><span class="s">&quot;jumbotron text-center&quot;</span><span class="nt">&gt;</span>Sparks<span class="w"> </span>AI<span class="nt">&lt;/h1&gt;</span>
<span class="w">        </span><span class="nt">&lt;div</span><span class="w"> </span><span class="na">class=</span><span class="s">&quot;chat-container&quot;</span><span class="nt">&gt;</span>
<span class="w">            </span><span class="nt">&lt;div</span><span class="w"> </span><span class="na">class=</span><span class="s">&quot;chat-box&quot;</span><span class="w"> </span><span class="na">id=</sp